In [1]:
import nltk
import numpy as np
import re
import string
import json # Потрібен для останнього завдання
from nltk.corpus import sentence_polarity
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

# Завантаження ресурсів
nltk.download('sentence_polarity')
nltk.download('stopwords')

# 1. Функція попередньої обробки (з Лаб 1)
def process_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # Видалення стилів Twitter (хоча dataset чистий, залишаємо для універсальності)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    text = re.sub(r'#', '', text)

    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tokens = tokenizer.tokenize(text)

    clean_tokens = []
    for word in tokens:
        if (word not in stopwords_english and word not in string.punctuation):
            stem_word = stemmer.stem(word)
            clean_tokens.append(stem_word)

    return clean_tokens

# 2. Завантаження даних (Варіант 7)
def get_data():
    # Склеюємо токени назад у речення для сумісності з process_text
    pos_sents = [" ".join(sent) for sent in sentence_polarity.sents(categories='pos')]
    neg_sents = [" ".join(sent) for sent in sentence_polarity.sents(categories='neg')]

    # Розділення 80/20
    split_pos = int(len(pos_sents) * 0.8)
    split_neg = int(len(neg_sents) * 0.8)

    train_x = pos_sents[:split_pos] + neg_sents[:split_neg]
    test_x = pos_sents[split_pos:] + neg_sents[split_neg:]

    # Мітки: 1 - позитив, 0 - негатив
    train_y = np.append(np.ones(len(pos_sents[:split_pos])), np.zeros(len(neg_sents[:split_neg])))
    test_y = np.append(np.ones(len(pos_sents[split_pos:])), np.zeros(len(neg_sents[split_neg:])))

    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = get_data()

# 3. Побудова частотного словника
def count_tweets(tweets, ys):
    result = {}
    for y, tweet in zip(ys, tweets):
        for word in process_text(tweet):
            pair = (word, y)
            result[pair] = result.get(pair, 0) + 1
    return result

freqs = count_tweets(train_x, train_y)
print(f"Кількість унікальних пар (слово, клас): {len(freqs)}")

[nltk_data] Downloading package sentence_polarity to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/sentence_polarity.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Кількість унікальних пар (слово, клас): 18449


In [2]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: словник частот (word, label): count
        train_x: список твітів
        train_y: список міток (0, 1)
    Output:
        logprior: логарифм відношення ймовірностей класів (скаляр)
        loglikelihood: словник {word: log_probability_ratio}
    '''
    loglikelihood = {}
    logprior = 0

    # Отримуємо унікальні слова (словник V)
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # Розрахунок N_pos та N_neg (загальна кількість слів у класах)
    N_pos = N_neg = 0
    for pair in freqs.keys():
        if pair[1] > 0:
            N_pos += freqs[pair]
        else:
            N_neg += freqs[pair]

    # Розрахунок кількості документів
    D = len(train_y)
    D_pos = sum(train_y) # Оскільки мітки 1, сума дасть кількість позитивних
    D_neg = D - D_pos

    # Обчислення Log Prior
    logprior = np.log(D_pos) - np.log(D_neg)

    # Обчислення Log Likelihood для кожного слова
    for word in vocab:
        freq_pos = freqs.get((word, 1), 0)
        freq_neg = freqs.get((word, 0), 0)

        # Згладжування Лапласа (+1)
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        loglikelihood[word] = np.log(p_w_pos / p_w_neg)

    return logprior, loglikelihood

# Тренування моделі
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(f"Log Prior: {logprior:.4f}")
# Примітка: Оскільки класи рівні (5331 vs 5331), logprior має бути 0.
print(f"Кількість слів у loglikelihood: {len(loglikelihood)}")

def naive_bayes_predict(tweet, logprior, loglikelihood):
    word_l = process_text(tweet)
    p = 0
    p += logprior

    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]

    return p

Log Prior: 0.0000
Кількість слів у loglikelihood: 13474


In [3]:
# 6. Оцінка точності
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    y_hats = []
    for tweet in test_x:
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            y_hats.append(1)
        else:
            y_hats.append(0)

    # Точність = середня кількість збігів
    accuracy = np.mean(np.array(y_hats) == test_y)
    return accuracy

acc = test_naive_bayes(test_x, test_y, logprior, loglikelihood)
print(f"Точність класифікатора (Accuracy): {acc:.4f}")

# 7. Аналіз співвідношень слів
def get_ratio(freqs, word):
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    pos_neg_ratio['positive'] = freqs.get((word, 1), 0)
    pos_neg_ratio['negative'] = freqs.get((word, 0), 0)

    # Розрахунок ratio з урахуванням згладжування (+1), щоб не ділити на 0
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1) / (pos_neg_ratio['negative'] + 1)
    return pos_neg_ratio

def get_words_by_threshold(freqs, label, threshold):
    word_list = {}
    for key in freqs.keys():
        word, _ = key
        # Щоб не рахувати одне слово двічі, беремо тільки один прохід
        if word not in word_list:
             pos_neg_ratio = get_ratio(freqs, word)
             if label == 1 and pos_neg_ratio['ratio'] >= threshold:
                 word_list[word] = pos_neg_ratio
             elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
                 word_list[word] = pos_neg_ratio
    return word_list

# Знайти найбільш позитивні слова (ratio > 10)
print("\n--- Top Positive Words ---")
pos_words = get_words_by_threshold(freqs, label=1, threshold=10)
for word, data in list(pos_words.items())[:5]: # Виведемо 5 прикладів
    print(f"Word: {word}, Ratio: {data['ratio']:.2f}")

# Знайти найбільш негативні слова (ratio < 0.2)
print("\n--- Top Negative Words ---")
neg_words = get_words_by_threshold(freqs, label=0, threshold=0.15)
for word, data in list(neg_words.items())[:5]:
    print(f"Word: {word}, Ratio: {data['ratio']:.2f}")

# 8. Аналіз помилок класифікації
print('\n--- Error Analysis (Examples) ---')
print('Label Pred  Tweet')
count = 0
for x, y in zip(test_x, test_y):
    pred_val = naive_bayes_predict(x, logprior, loglikelihood)
    y_hat = 1 if pred_val > 0 else 0

    if y != y_hat:
        print(f'{int(y)}     {y_hat}     {x[:60]}...') # Показуємо перші 60 символів
        count += 1
        if count >= 5: break # Показати лише 5 помилок

Точність класифікатора (Accuracy): 0.7741

--- Top Positive Words ---
Word: ingeni, Ratio: 10.00
Word: rivet, Ratio: 18.00
Word: refresh, Ratio: 11.00
Word: engross, Ratio: 14.00
Word: russian, Ratio: 10.00

--- Top Negative Words ---
Word: dull, Ratio: 0.11
Word: bad, Ratio: 0.14
Word: bore, Ratio: 0.09
Word: bland, Ratio: 0.11
Word: stupid, Ratio: 0.12

--- Error Analysis (Examples) ---
Label Pred  Tweet
1     0     the soundtrack alone is worth the price of admission ....
1     0     the importance of being earnest , so thick with wit it plays...
1     0     made for teens and reviewed as such , this is recommended on...
1     0     the film does give a pretty good overall picture of the situ...
1     0     nothing short of wonderful with its ten-year-old female prot...
